In [1]:
import pandas as pd

In [2]:
servers = pd.DataFrame()
servers['name'] = [f'PC{i:02}' for i in range(1, 5)] + \
                [f'PC{i:02}' for i in range(6, 8)] + \
                [f'PC{i:02}' for i in range(9, 11)]
servers['hostname'] = ['172.19.4.225', '172.19.4.204', '172.19.4.235',
                 '172.19.4.200', '172.19.4.215', '172.19.4.238',
                 '172.19.4.203', '172.19.4.234']
servers['username'] = ['robert'] + ['iapucp'] * 7
servers['password'] = ['291295'] + ['iapucp504'] * 5 + \
                      ['intart504'] + ['iapucp504']
servers['model'] = ['HP i7'] * 3 + ['MSI'] + ['HP i7'] + \
                   ['MSI'] * 2 + ['Lenovo i5']


In [3]:
servers

,name,hostname,username,password,model
0,PC01,172.19.4.225,robert,291295,HP i7
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7
3,PC04,172.19.4.200,iapucp,iapucp504,MSI
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7
5,PC07,172.19.4.238,iapucp,iapucp504,MSI
6,PC09,172.19.4.203,iapucp,intart504,MSI
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5


In [ ]:
cd Desktop/Experiments && bash setup.sh

In [62]:
servers.to_csv('servers.csv', index=False)

In [4]:
import paramiko
def connect(servers, name):
    server = servers[servers.name==name]
    params = server[['hostname', 'username', 'password']] \
                .iloc[0].to_dict()
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy()) 
    ssh.connect(**params)
    return ssh

In [7]:
ssh = connect(servers, 'PC01')
stdin, stdout, stderr = ssh.exec_command('ls Desktop/Experiments')
for line in stdout: print(line, end='') 

Bags
CTNNB1-Raw-BCR.txt
CTNNB1-Raw-CD40.txt
DataCV
Dockerfile
environment.yml
FeatRanking
run_experiment.py


In [43]:
ssh = connect(servers, 'PC01')
stdin, stdout, stderr = ssh.exec_command('ls Desktop/Experiments/Results -lha')
for line in stdout: print(line, end='') 

PC01
total 1,2M
drwxrwxr-x 2 robert robert 4,0K sit 11 00:22 .
drwxrwxr-x 6 robert robert 4,0K sit 11 00:17 ..
-rw-rw-r-- 1 robert robert 1,2M sit 11 14:47 CTNNB1-Raw-All.txt


In [67]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    ssh.exec_command('mkdir -p ~/Desktop/Experiments')

CPU times: user 248 ms, sys: 8 ms, total: 256 ms
Wall time: 2.46 s


In [37]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    ssh.exec_command('mkdir -p ~/Desktop/Experiments/Results')

CPU times: user 232 ms, sys: 20 ms, total: 252 ms
Wall time: 1.86 s


In [ ]:
import os
os.chdir('..')

In [5]:
from scp import SCPClient

In [15]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport())
    scp.put(['environment.yml', 'setup.sh'], 
            remote_path='~/Desktop/Experiments')

CPU times: user 188 ms, sys: 24 ms, total: 212 ms
Wall time: 2.6 s


## Transport Bags and FeatRanking

In [18]:
from tqdm import tqdm_notebook

In [20]:
%%time
for server in tqdm_notebook(servers.name):
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport())
    scp.put([path + 'Bags', path + 'FeatRanking'], 
            remote_path='~/Desktop/Experiments', recursive=True)

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))


CPU times: user 7.88 s, sys: 592 ms, total: 8.47 s
Wall time: 48.1 s


In [22]:
def progress(filename, size, sent):
    print("\r%s\'s progress: %.2f%% " % \
          (filename, float(sent)/float(size)*100), end='')

In [25]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    ssh.exec_command('mkdir -p ~/Desktop/Experiments/DataCV')

CPU times: user 252 ms, sys: 16 ms, total: 268 ms
Wall time: 1.84 s


In [26]:
path = '_data/genomics-data/DataCV/'

In [ ]:
%%time
for server in tqdm_notebook(servers.name[1:], total=len(servers)-1):
    print(server)
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport(), progress = progress)
    scp.put([path + 'Dc', path + 'Di', path + 'Raw'], 
            remote_path='~/Desktop/Experiments/DataCV', 
            recursive=True)

HBox(children=(IntProgress(value=0, max=7), HTML(value='')))

PC02
b'Erk_CD40(0.01).tst.mat''s progress: 100.00% PC03
b'Jnk_BCR(0.01).tst.mat''s progress: 61.63% 

## Transport script

In [82]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport())
    scp.put(['run_experiment.py'], 
            remote_path='~/Desktop/Experiments')

CPU times: user 240 ms, sys: 92 ms, total: 332 ms
Wall time: 2.25 s


In [36]:
servers

,name,hostname,username,password,model
0,PC01,172.19.4.225,robert,291295,HP i7
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7
3,PC04,172.19.4.200,iapucp,iapucp504,MSI
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7
5,PC07,172.19.4.238,iapucp,iapucp504,MSI
6,PC09,172.19.4.203,iapucp,intart504,MSI
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5


In [ ]:
sudo apt-get install screen

In [ ]:
screen
source activate iapucp
cd Desktop/Experiments
python run_experiment.py IRF4 Di BCR

## Monitor progress

In [85]:
%%time
for server in servers.name:
    print(f'{server}: ', end='')
    ssh = connect(servers, server)
    stdin, stdout, stderr = ssh.exec_command('ls Des\ktop/Experiments/Results -lht')
    for line in stdout: print(line, end='')

PC01: total 11M
-rw-rw-r-- 1 robert robert 896K sit 19 16:49 MYC-Raw-BCR.txt
-rw-rw-r-- 1 robert robert 2,0M sit 17 01:30 LEF1-Raw-CD40.txt
-rw-rw-r-- 1 robert robert 2,1M sit 16 07:09 Jnk-Raw-All.txt
-rw-rw-r-- 1 robert robert 2,1M sit 14 05:33 Ikk2-Di-BCR.txt
-rw-rw-r-- 1 robert robert 2,0M sit 12 05:42 IRF4-Di-CD40.txt
-rw-rw-r-- 1 robert robert 2,0M sit 11 20:23 CTNNB1-Raw-All.txt
PC02: total 11M
-rw-rw-r-- 1 iapucp iapucp 905K sit 19 16:49 MYC-Raw-CD40.txt
-rw-rw-r-- 1 iapucp iapucp 2,0M sit 17 12:09 LEF1-Raw-All.txt
-rw-rw-r-- 1 iapucp iapucp 2,1M sit 15 12:19 Jnk-Di-BCR.txt
-rw-rw-r-- 1 iapucp iapucp 2,0M sit 14 05:36 Ikk2-Di-CD40.txt
-rw-rw-r-- 1 iapucp iapucp 2,0M sit 12 23:03 IRF4-Di-All.txt
-rw-rw-r-- 1 iapucp iapucp 2,0M sit 11 05:56 CTNNB1-Di-BCR.txt
PC03: total 11M
-rw-rw-r-- 1 iapucp iapucp 856K sit 19 16:49 MYC-Raw-All.txt
-rw-rw-r-- 1 iapucp iapucp 2,0M sit 17 02:07 LEF1-Di-BCR.txt
-rw-rw-r-- 1 iapucp iapucp 2,1M sit 15 12:38 Jnk-Di-CD40.txt
-rw-rw-r-- 1 iapucp iapucp 

## Prepare next experiments

In [52]:
experiments = servers.copy()

In [54]:
experiments['Dataset'] = ['IRF4']*5 + ['Ikk2']*3
experiments['Featureset'] = ['Di']*2 + ['Dc']*3 + ['Raw']*3
experiments['Treatment'] = (['BCR', 'CD40', 'All']*3)[1:]

In [55]:
experiments

,name,hostname,username,password,model,Dataset,Featureset,Treatment
0,PC01,172.19.4.225,robert,291295,HP i7,IRF4,Di,CD40
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7,IRF4,Di,All
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7,IRF4,Dc,BCR
3,PC04,172.19.4.200,iapucp,iapucp504,MSI,IRF4,Dc,CD40
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7,IRF4,Dc,All
5,PC07,172.19.4.238,iapucp,iapucp504,MSI,Ikk2,Raw,BCR
6,PC09,172.19.4.203,iapucp,intart504,MSI,Ikk2,Raw,CD40
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5,Ikk2,Raw,All


In [65]:
experiments = servers.copy()
experiments['Dataset'] = ['Ikk2']*6 + ['Jnk']*2
experiments['Featureset'] = ['Di']*3 + ['Dc']*3 + ['Raw']*2
experiments['Treatment'] = (['BCR', 'CD40', 'All']*3)[:-1]
experiments

,name,hostname,username,password,model,Dataset,Featureset,Treatment
0,PC01,172.19.4.225,robert,291295,HP i7,Ikk2,Di,BCR
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7,Ikk2,Di,CD40
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7,Ikk2,Di,All
3,PC04,172.19.4.200,iapucp,iapucp504,MSI,Ikk2,Dc,BCR
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7,Ikk2,Dc,CD40
5,PC07,172.19.4.238,iapucp,iapucp504,MSI,Ikk2,Dc,All
6,PC09,172.19.4.203,iapucp,intart504,MSI,Jnk,Raw,BCR
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5,Jnk,Raw,CD40


In [69]:
experiments = servers.copy()
experiments['Dataset'] = ['Jnk']*7 + ['LEF1']
experiments['Featureset'] = ['Raw'] + ['Di']*3 + ['Dc']*3 + ['Raw']
experiments['Treatment'] = (['All', 'BCR', 'CD40']*3)[:-1]
experiments

,name,hostname,username,password,model,Dataset,Featureset,Treatment
0,PC01,172.19.4.225,robert,291295,HP i7,Jnk,Raw,All
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7,Jnk,Di,BCR
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7,Jnk,Di,CD40
3,PC04,172.19.4.200,iapucp,iapucp504,MSI,Jnk,Di,All
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7,Jnk,Dc,BCR
5,PC07,172.19.4.238,iapucp,iapucp504,MSI,Jnk,Dc,CD40
6,PC09,172.19.4.203,iapucp,intart504,MSI,Jnk,Dc,All
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5,LEF1,Raw,BCR


In [76]:
experiments = servers.copy()
experiments['Dataset'] = ['LEF1']*8
experiments['Featureset'] = ['Raw']*2 + ['Di']*3 + ['Dc']*3
experiments['Treatment'] = (['CD40', 'All', 'BCR']*3)[:-1]
experiments

,name,hostname,username,password,model,Dataset,Featureset,Treatment
0,PC01,172.19.4.225,robert,291295,HP i7,LEF1,Raw,CD40
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7,LEF1,Raw,All
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7,LEF1,Di,BCR
3,PC04,172.19.4.200,iapucp,iapucp504,MSI,LEF1,Di,CD40
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7,LEF1,Di,All
5,PC07,172.19.4.238,iapucp,iapucp504,MSI,LEF1,Dc,BCR
6,PC09,172.19.4.203,iapucp,intart504,MSI,LEF1,Dc,CD40
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5,LEF1,Dc,All


In [79]:
experiments = servers.copy()
experiments['Dataset'] = ['MYC']*8
experiments['Featureset'] = ['Raw']*3 + ['Di']*3 + ['Dc']*2
experiments['Treatment'] = (['BCR', 'CD40', 'All']*3)[:-1]
experiments

,name,hostname,username,password,model,Dataset,Featureset,Treatment
0,PC01,172.19.4.225,robert,291295,HP i7,MYC,Raw,BCR
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7,MYC,Raw,CD40
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7,MYC,Raw,All
3,PC04,172.19.4.200,iapucp,iapucp504,MSI,MYC,Di,BCR
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7,MYC,Di,CD40
5,PC07,172.19.4.238,iapucp,iapucp504,MSI,MYC,Di,All
6,PC09,172.19.4.203,iapucp,intart504,MSI,MYC,Dc,BCR
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5,MYC,Dc,CD40


In [80]:
for idx, row in experiments.iterrows():
    print(f"sshpass -p {row['password']} "
          f"ssh {row['username']}@{row['hostname']}")
#     print('cd ~/Desktop/Experiments/ && source activate iapucp')
    print(f"python run_experiment.py {row['Dataset']} "
          f"{row['Featureset']} {row['Treatment']}")
    print()

sshpass -p 291295 ssh robert@172.19.4.225
python run_experiment.py MYC Raw BCR

sshpass -p iapucp504 ssh iapucp@172.19.4.204
python run_experiment.py MYC Raw CD40

sshpass -p iapucp504 ssh iapucp@172.19.4.235
python run_experiment.py MYC Raw All

sshpass -p iapucp504 ssh iapucp@172.19.4.200
python run_experiment.py MYC Di BCR

sshpass -p iapucp504 ssh iapucp@172.19.4.215
python run_experiment.py MYC Di CD40

sshpass -p iapucp504 ssh iapucp@172.19.4.238
python run_experiment.py MYC Di All

sshpass -p intart504 ssh iapucp@172.19.4.203
python run_experiment.py MYC Dc BCR

sshpass -p iapucp504 ssh iapucp@172.19.4.234
python run_experiment.py MYC Dc CD40



## Retrieve results

In [84]:
for server in tqdm_notebook(servers.name, total=len(servers)):
    print(server)
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport(), progress=progress)
    scp.get('~/Desktop/Experiments/Results/', './Results/',
            recursive=True, preserve_times=True)

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))

PC01
b'./Results/Results/IRF4-Di-CD40.txt''s progress: 100.00% PC02
b'./Results/Results/MYC-Raw-CD40.txt''s progress: 100.00% PC03
b'./Results/Results/Ikk2-Di-All.txt''s progress: 100.00% PC04
b'./Results/Results/Jnk-Di-All.txt''s progress: 100.00% PC06
b'./Results/Results/Jnk-Dc-BCR.txt''s progress: 100.00% PC07
b'./Results/Results/LEF1-Dc-BCR.txt''s progress: 100.00% PC09
b'./Results/Results/Jnk-Dc-All.txt''s progress: 100.00% PC10
b'./Results/Results/Ikk2-Raw-All.txt''s progress: 100.00% 
